<a href="https://colab.research.google.com/github/OleDziewas/MotionDiffusion/blob/main/PyMoTests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/youngwoo-yoon/PyMO.git

Cloning into 'PyMO'...
remote: Enumerating objects: 199, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 199 (delta 40), reused 38 (delta 37), pack-reused 150
Receiving objects: 100% (199/199), 6.05 MiB | 15.45 MiB/s, done.
Resolving deltas: 100% (99/99), done.


In [ ]:
%cd PyMO

/content/PyMO


In [1]:
!mkdir /content/val
!unzip /content/drive/MyDrive/GENEADataset/val/bvh.zip -d /content/val
#!unzip /content/drive/MyDrive/GENEADataset/val/tsv.zip -d /content/val
#!unzip /content/drive/MyDrive/GENEADataset/trn/bvh.zip -d /content/trn
#!unzip /content/drive/MyDrive/GENEADataset/trn/tsv.zip -d /content/trn
#!unzip /content/drive/MyDrive/GENEADataset/test/tsv.zip -d /content/tsv

Archive:  /content/drive/MyDrive/GENEADataset/val.zip
   creating: /content/val/
  inflating: /content/val/wav.zip    

In [ ]:
!mkdir /content/input

In [ ]:
!cp /content/val/bvh/val_2022_v1_000.bvh /content/input

# BVH2Feature


In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

import glob
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import IPython

from pymo.parsers import BVHParser
from pymo.data import Joint, MocapData
from pymo.preprocessing import *
from pymo.viz_tools import *
from pymo.writers import *

import joblib as jl

In [ ]:
def extract_joint_angles(bvh_dir, files, destpath, fps):

    if not os.path.exists(destpath):
         os.mkdir(destpath)

    p = BVHParser()

    data_all = list()
    for f in files:
        ff = os.path.join(bvh_dir, f + '.bvh')
        print(ff)
        data_all.append(p.parse(ff))

    # 24 joints (upper and lower body excluding fingers)
    target_joints = ['body_world', 'b_root', 'b_l_upleg', 'b_l_leg', 'b_r_upleg', 'b_r_leg', 'b_spine0', 'b_spine1', 'b_spine2', 'b_spine3', 'b_l_shoulder', 'b_l_arm', 'b_l_arm_twist', 'b_l_forearm', 'b_l_wrist_twist', 'b_l_wrist', 'b_r_shoulder', 'b_r_arm', 'b_r_arm_twist', 'b_r_forearm', 'b_r_wrist_twist', 'b_r_wrist', 'b_neck0', 'b_head']

    data_pipe = Pipeline([
       ('dwnsampl', DownSampler(tgt_fps=30,  keep_all=False)),
       ('root', RootNormalizer()),
       ('jtsel', JointSelector(target_joints, include_root=False)),
       ('exp', MocapParameterizer('expmap')),
       ('np', Numpyfier())
    ])


    out_data = data_pipe.fit_transform(data_all)
    #cnt = 0
    #for i in range(len(out_data)):
    #  for e in range(len(out_data[i])):
    #    for q in range(len(out_data[i][e])):
    #      out_data[i][e][q] = cnt
    #      cnt+=1

    # the datapipe will append the mirrored files to the end
    #assert len(out_data) == 2*len(files)

    jl.dump(data_pipe, os.path.join(destpath, 'data_pipe.sav'))

    fi=0
    for f in files:
        ff = os.path.join(destpath, f)
        print(ff)
        np.savez(ff + ".npz", clips=out_data[fi])
        fi=fi+1

    return out_data, data_all

In [ ]:
# r=root, d=directories, f = files
dest_dir = '/content/output'
bvh_dir = '/content/input'

# list all bvh files in the bvh_dir
files = []
for r, d, f in os.walk(bvh_dir):
    for file in f:
        print(file)
        if '.bvh' in file:
            ff=os.path.join(r, file)
            basename = os.path.splitext(os.path.basename(ff))[0]
            files.append(basename)

# process to destpath
out_data, data_all = extract_joint_angles(bvh_dir, files, dest_dir, fps=20)



val_2022_v1_000.bvh
/content/input/val_2022_v1_000.bvh
JointSelector
MocapParameterizer: expmap
Numpyfier
/content/output/val_2022_v1_000


In [ ]:
out_data.shape

(1, 1890, 6)

# Feature2Bvh

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from pymo.parsers import BVHParser
from pymo.data import Joint, MocapData
from pymo.preprocessing import *
from pymo.viz_tools import *
from pymo.writers import *

import joblib as jl

# load data pipeline
pipeline = jl.load('/content/output/data_pipe.sav')
features = np.load("/content/output/val_2022_v1_000.npz")['clips']
print(features.shape)

# transform the data back to it's original shape
# note: in a real scenario this is usually done with predicted data
# note: some transformations (such as transforming to joint positions) are not inversible
bvh_data=pipeline.inverse_transform([features])

# Test to write some of it to file for visualization in blender or motion builder
writer = BVHWriter()
with open('/content/output/converted.bvh','w') as f:
    writer.write(bvh_data[0], f)

In [ ]:
!diff /content/output/converted.bvh /content/PyMO/demos/data/AV_8Walk_Meredith_HVHA_Rep1.bvh -y